# Importing csv files into postgres database

## Steps

- import csv into pandas dataframe
- explore for potential inconsistencies with database schema
- clean the table data where needed e.g. change data types
- alter database schema where needed
- import file into db

In [1]:
#Import libraries and modules

import os
import numpy as np
import pandas as pd
import psycopg2
import datetime
import csv
from jproperties import Properties


In [54]:
#Importing csv files into pandas dataframes
#replace empty string with path to the file

Person = pd.read_csv("social_network/person_0_0.csv", sep='|')
PersonEmail = pd.read_csv("social_network/person_email_emailaddress_0_0.csv", sep='|')
PersonLanguage = ""
Organizations = pd.read_csv("social_network/organisation_0_0.csv", sep='|')
PersonStudyAtUniversity = pd.read_csv("social_network/person_studyAt_organisation_0_0.csv", sep='|')
PersonWorkAtCompany = pd.read_csv("social_network/person_workAt_organisation_0_0.csv", sep='|')
Forum = pd.read_csv("social_network/forum_0_0.csv", sep='|')
Post = pd.read_csv("social_network/post_0_0.csv", sep='|')
Message = ""
Comment = ""
Tag = pd.read_csv("social_network/tag_0_0.csv", sep='|')
TagClass = pd.read_csv("social_network/tagclass_0_0.csv", sep='|')
PostHasTag = pd.read_csv("social_network/post_hasTag_tag_0_0.csv", sep='|')
TagHasType = pd.read_csv("social_network/tag_hasType_tagclass_0_0.csv", sep='|')
Place = pd.read_csv("social_network/place_0_0.csv", sep='|')
PersonKnowsPerson = pd.read_csv("social_network/person_knows_person_0_0.csv", sep='|')
ForumHasMember = pd.read_csv("social_network/forum_hasMember_person_0_0.csv", sep='|')



## Person Table

In [12]:
Person.head()

,id,firstName,lastName,gender,birthday,creationDate,locationIP,browserUsed,place
0,12094627905604,Mirza Kalich,Ali,male,1989-11-02,2012-02-04T06:37:59.956+0000,175.111.0.55,Firefox,794
1,2199023255625,Cheng,Chen,male,1981-12-06,2010-07-12T15:33:44.057+0000,27.98.233.59,Internet Explorer,330
2,2199023255600,Adje van den Berg,Vries,female,1986-08-07,2010-07-14T22:01:21.407+0000,192.101.114.157,Firefox,1253
3,16492674416673,Ali,Ferrer,female,1981-06-30,2012-08-26T07:34:09.342+0000,195.39.218.25,Safari,973
4,2199023255611,Chen,Yang,female,1982-08-28,2010-07-24T07:48:35.041+0000,1.2.2.172,Firefox,427


In [14]:
#For the first table, I replaced dataframe headers, 
#but then realized it was unnecessary because we are just importing the rows (This would ne necessary if creating tables)
#So this step is skipped for the rest of the tables

#replacing headers

"""
CREATE TABLE Person (
    PersonID SERIAL PRIMARY KEY,
    CreationDate TIMESTAMP NOT NULL,
    FirstName VARCHAR(255),
    LastName VARCHAR(255),
    Gender VARCHAR(50),
    Birthday DATE CHECK (Birthday <= CURRENT_DATE),
    BrowserUsed VARCHAR(255),
    LocationIP VARCHAR(255)
);
"""
new_columns = {
    
    "id": "PersonID",
    "creationDate": "CreationDate",
    "firstName": "FirstName",
    "lastName": "LastName",
    "gender": "Gender",
    "birthday": "Birthday",
    "locationIP": "LocationIP",
    "browserUsed": "BrowserUsed",
    "place": "Place" #what is place
}

Person = Person.rename(columns=new_columns)

Person.head(2)

,PersonID,FirstName,LastName,Gender,Birthday,CreationDate,LocationIP,BrowserUsed,Place
0,12094627905604,Mirza Kalich,Ali,male,1989-11-02,2012-02-04T06:37:59.956+0000,175.111.0.55,Firefox,794
1,2199023255625,Cheng,Chen,male,1981-12-06,2010-07-12T15:33:44.057+0000,27.98.233.59,Internet Explorer,330


In [15]:
#Datatypes
Person.info()


"""
CREATE TABLE Person (
    PersonID BIGINT PRIMARY KEY,
    CreationDate TIMESTAMP NOT NULL,
    FirstName VARCHAR(255),
    LastName VARCHAR(255),
    Gender VARCHAR(50),
    Birthday DATE CHECK (Birthday <= CURRENT_DATE),
    BrowserUsed VARCHAR(255),
    LocationIP VARCHAR(255)
);
"""

# TODO
#   CreationDate to from object to datetime - done
#   Birthday to datetime - done

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PersonID      88 non-null     int64 
 1   FirstName     88 non-null     object
 2   LastName      88 non-null     object
 3   Gender        88 non-null     object
 4   Birthday      88 non-null     object
 5   CreationDate  88 non-null     object
 6   LocationIP    88 non-null     object
 7   BrowserUsed   88 non-null     object
 8   Place         88 non-null     int64 
dtypes: int64(2), object(7)
memory usage: 6.3+ KB


In [16]:
Person['CreationDate'] = pd.to_datetime(Person['CreationDate'], errors='coerce')


In [18]:
Person['Birthday'] = pd.to_datetime(Person['Birthday'], errors='coerce')


In [19]:
Person.info()

#datatypes ready for import

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   PersonID      88 non-null     int64              
 1   FirstName     88 non-null     object             
 2   LastName      88 non-null     object             
 3   Gender        88 non-null     object             
 4   Birthday      88 non-null     datetime64[ns]     
 5   CreationDate  88 non-null     datetime64[ns, UTC]
 6   LocationIP    88 non-null     object             
 7   BrowserUsed   88 non-null     object             
 8   Place         88 non-null     int64              
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), int64(2), object(5)
memory usage: 6.3+ KB


In [26]:
#insert data

query = """INSERT INTO Person (
    PersonID, 
    CreationDate, 
    FirstName, 
    LastName, 
    Gender, 
    Birthday, 
    BrowserUsed, 
    LocationIP) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""



#create connection by replacing the database dtails with your database info

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in Person.iterrows():
        # Extract values from the DataFrame
        PersonID_value = row['PersonID']
        CreationDate_value = row['CreationDate']
        FirstName_value = row['FirstName']
        LastName_value = row['LastName']
        Gender_value = row['Gender']
        Birthday_value = row['Birthday']
        BrowserUsed_value = row['BrowserUsed']
        LocationIP_value = row['LocationIP']
        
        # Execute the SQL query
        cursor.execute(query, (PersonID_value,
                               CreationDate_value,
                               FirstName_value,
                               LastName_value,
                               Gender_value,
                               Birthday_value,
                               BrowserUsed_value,
                               LocationIP_value))
        connection.commit()

# Close the database connection
connection.close()

## PersonEmail

In [29]:
PersonEmail.head()

,Person.id,email
0,12094627905604,Mirza.Kalich12094627905604@gmail.com
1,12094627905604,Mirza.Kalich12094627905604@gmx.com
2,12094627905604,Mirza.Kalich12094627905604@zoho.com
3,2199023255625,Cheng2199023255625@gmail.com
4,2199023255600,Adje.van.den.Berg2199023255600@gmail.com


In [30]:
PersonEmail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Person.id  152 non-null    int64 
 1   email      152 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [33]:
columns = {
    
    "Person.id": "PersonID",
    "email": "Email",
}

PersonEmail = PersonEmail.rename(columns=columns)

PersonEmail.head(2)

,PersonID,Email
0,12094627905604,Mirza.Kalich12094627905604@gmail.com
1,12094627905604,Mirza.Kalich12094627905604@gmx.com


In [36]:
#insert data

query = """INSERT INTO PersonEmail (
    PersonID, 
    Email) 
    VALUES (%s, %s)"""



connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in PersonEmail.iterrows():
        # Extract values from the DataFrame
        PersonID_value = row['PersonID']
        Email_value = row['Email']
        
        # Execute the SQL query
        cursor.execute(query, (PersonID_value,
                               Email_value))
        connection.commit()


connection.close()

## University/Company

In [38]:
Organizations.head()

,id,type,name,url,place
0,0,company,Kam_Air,http://dbpedia.org/resource/Kam_Air,59
1,1,company,Balkh_Airlines,http://dbpedia.org/resource/Balkh_Airlines,59
2,2,company,Khyber_Afghan_Airlines,http://dbpedia.org/resource/Khyber_Afghan_Airl...,59
3,3,company,MarcoPolo_Airways,http://dbpedia.org/resource/MarcoPolo_Airways,59
4,4,company,Pamir_Airways,http://dbpedia.org/resource/Pamir_Airways,59


In [39]:
Organizations['type'].unique()

array(['company', 'university'], dtype=object)

In [ ]:
"""
-- Create University table
CREATE TABLE University (
    UniversityID INT PRIMARY KEY,
    Name VARCHAR(255) NOT NULL
);

-- Create Company table
CREATE TABLE Company (
    CompanyID INT PRIMARY KEY,
    Name VARCHAR(255) NOT NULL
);

"""

In [40]:
company_filter = Organizations['type'] == 'company'
university_filter = Organizations['type'] == 'university'

company = Organizations.loc[company_filter]
university = Organizations.loc[university_filter]

In [41]:
company.head()

,id,type,name,url,place
0,0,company,Kam_Air,http://dbpedia.org/resource/Kam_Air,59
1,1,company,Balkh_Airlines,http://dbpedia.org/resource/Balkh_Airlines,59
2,2,company,Khyber_Afghan_Airlines,http://dbpedia.org/resource/Khyber_Afghan_Airl...,59
3,3,company,MarcoPolo_Airways,http://dbpedia.org/resource/MarcoPolo_Airways,59
4,4,company,Pamir_Airways,http://dbpedia.org/resource/Pamir_Airways,59


In [42]:
university.head()

,id,type,name,url,place
1575,1575,university,Paktia_University,http://dbpedia.org/resource/Paktia_University,961
1576,1576,university,Baghlan_University,http://dbpedia.org/resource/Baghlan_University,955
1577,1577,university,Syed_Jamaluddin_Afghan_University,http://dbpedia.org/resource/Syed_Jamaluddin_Af...,958
1578,1578,university,Bost_University,http://dbpedia.org/resource/Bost_University,956
1579,1579,university,Aga_Khan_University,http://dbpedia.org/resource/Aga_Khan_University,775


In [47]:
#insert company data

query = """INSERT INTO Company (
    CompanyID, 
    Name,
    URL) 
    VALUES (%s, %s, %s)"""



connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in company.iterrows():
        # Extract values from the DataFrame
        CompanyID_value = row['id']
        Name_value = row['name']
        URL_value = row['url']
        
        # Execute the SQL query
        cursor.execute(query, (CompanyID_value,
                               Name_value,
                              URL_value))
        connection.commit()

# Close the database connection
connection.close()

In [49]:
#insert university data

query = """INSERT INTO University (
    UniversityID, 
    Name,
    URL) 
    VALUES (%s, %s, %s)"""



# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in university.iterrows():
        # Extract values from the DataFrame
        CompanyID_value = row['id']
        Name_value = row['name']
        URL_value = row['url']
        
        # Execute the SQL query
        cursor.execute(query, (CompanyID_value,
                               Name_value,
                              URL_value))
        connection.commit()


connection.close()

# PersonStudyAtUniversity

In [53]:
PersonStudyAtUniversity.head()

,Person.id,Organisation.id,classYear
0,2199023255625,2207,2002
1,16492674416673,1596,2002
2,2199023255611,2207,2001
3,13194139533352,1856,2007
4,3298534883391,2833,2005


In [54]:
PersonStudyAtUniversity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Person.id        71 non-null     int64
 1   Organisation.id  71 non-null     int64
 2   classYear        71 non-null     int64
dtypes: int64(3)
memory usage: 1.8 KB


In [5]:
#insert personstudyatuniversity

from psycopg2.extensions import register_adapter, AsIs
register_adapter(np.int64, AsIs)

query = """INSERT INTO PersonStudyAtUniversity (
    PersonID, 
    UniversityID,
    ClassYear) 
    VALUES (%s, %s, %s)"""



# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in PersonStudyAtUniversity.iterrows():
        # Extract values from the DataFrame
        PersonID_value = row['Person.id']
        UniversityID_value = row['Organisation.id']
        ClassYear_value = row['classYear']
        
        
        
        # Execute the SQL query
        cursor.execute(query, (PersonID_value,
                               UniversityID_value,
                              ClassYear_value))
        connection.commit()

# Close the database connection
connection.close()

## PersonWorkAtCompany

In [7]:
PersonWorkAtCompany.head()

,Person.id,Organisation.id,workFrom
0,12094627905604,897,2009
1,12094627905604,887,2008
2,12094627905604,888,2012
3,12094627905604,890,2011
4,12094627905604,893,2009


In [8]:
query = """INSERT INTO PersonWorkAtCompany (
    PersonID, 
    CompanyID,
    WorkFrom) 
    VALUES (%s, %s, %s)"""



# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in PersonWorkAtCompany.iterrows():
        # Extract values from the DataFrame
        PersonID_value = row['Person.id']
        CompanyID_value = row['Organisation.id']
        WorkFrom_value = row['workFrom']
        
        
        
        # Execute the SQL query
        cursor.execute(query, (PersonID_value,
                               CompanyID_value,
                              WorkFrom_value))
        connection.commit()

# Close the database connection
connection.close()

## Forum Table

In [11]:
"""
CREATE TABLE Forum (
    ForumID INT PRIMARY KEY,
    Title VARCHAR(255) NOT NULL,
    CreateDate TIMESTAMP NOT NULL
);
"""
Forum.info()

#change forumid to bigint -DONE
#change creationdate to datetime -DONE
#moderator column references personid -done

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            799 non-null    int64 
 1   title         799 non-null    object
 2   creationDate  799 non-null    object
 3   moderator     799 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 25.1+ KB


In [13]:
Forum['creationDate'] = pd.to_datetime(Forum['creationDate'], errors='coerce')

In [14]:
Forum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   id            799 non-null    int64              
 1   title         799 non-null    object             
 2   creationDate  799 non-null    datetime64[ns, UTC]
 3   moderator     799 non-null    int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(1)
memory usage: 25.1+ KB


In [15]:
query = """INSERT INTO Forum (
    ForumID, 
    Title,
    CreateDate,
    Moderator) 
    VALUES (%s, %s, %s, %s)"""


# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in Forum.iterrows():
        # Extract values from the DataFrame
        ForumID_value = row['id']
        Title_value = row['title']
        CreationDate_value = row['creationDate']
        Moderator_value = row['moderator']
        
        
        cursor.execute(query, (ForumID_value,
                               Title_value,
                              CreationDate_value,
                              Moderator_value))
        connection.commit()

# Close the database connection
connection.close()

## Message Table

In [ ]:
"""
CREATE TABLE Message (
    MessageID INT PRIMARY KEY,
    CreationDate TIMESTAMP NOT NULL,
    BrowserUsed VARCHAR(255),
    LocationIP VARCHAR(255),
    Content TEXT,
    Length INT,
    PersonID INT NOT NULL,
    FOREIGN KEY (PersonID) REFERENCES Person(PersonID) ON DELETE CASCADE
);
"""

## Person knows person

In [20]:
"""
CREATE TABLE PersonKnowsPerson (
    PersonID_A INT NOT NULL,
    PersonID_B INT NOT NULL,
    PRIMARY KEY (PersonID_A, PersonID_B),
    FOREIGN KEY (PersonID_A) REFERENCES Person(PersonID) ON DELETE CASCADE,
    FOREIGN KEY (PersonID_B) REFERENCES Person(PersonID) ON DELETE CASCADE
);
"""
PersonKnowsPerson.head()

#TODO
#change both personid columns to bigint - DONE
#add creationdatecolumn - DONE
#change creationdate in dataframe to datetime

,Person.id,Person.id.1,creationDate
0,2199023255625,9895604649984,2011-09-15T05:28:23.616+0000
1,2199023255625,8796093022217,2011-07-26T03:59:52.456+0000
2,2199023255625,15393162788888,2012-07-11T04:36:32.044+0000
3,2199023255625,9895604650020,2011-10-18T05:30:28.405+0000
4,2199023255625,16492674416689,2012-09-11T04:07:15.731+0000


In [21]:
PersonKnowsPerson['creationDate'] = pd.to_datetime(PersonKnowsPerson['creationDate'], errors='coerce')

In [22]:
PersonKnowsPerson.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Person.id     224 non-null    int64              
 1   Person.id.1   224 non-null    int64              
 2   creationDate  224 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(2)
memory usage: 5.4 KB


In [25]:
#inser into personknowsperson
query = """INSERT INTO PersonKnowsPerson (
    PersonID_A, 
    PersonID_B,
    CreationDate) 
    VALUES (%s, %s, %s)"""


# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in PersonKnowsPerson.iterrows():
        # Extract values from the DataFrame
        PersonID_A_value = row['Person.id']
        PersonID_B_value = row['Person.id.1']
        CreationDate_value = row['creationDate']

        
        
        cursor.execute(query, (PersonID_A_value,
                               PersonID_B_value,
                              CreationDate_value))
        connection.commit()

# Close the database connection
connection.close()

## Forum Has Member 

In [27]:
ForumHasMember.head()

"""
CREATE TABLE ForumHasMember (
    ForumID INT NOT NULL,
    PersonID INT NOT NULL,
    JoinDate TIMESTAMP NOT NULL,
"""

#TODO change forumid to bigint in db - Done
#TODO change personid to bigint in db - Done
#Change joinDate to datetime in df

,Forum.id,Person.id,joinDate
0,17179869197,9895604649984,2011-09-15T05:28:33.616+0000
1,17179869197,8796093022217,2011-07-26T04:00:02.456+0000
2,17179869197,15393162788888,2012-07-11T04:36:42.044+0000
3,17179869197,9895604650020,2011-10-18T05:30:38.405+0000
4,17179869197,16492674416689,2012-09-11T04:07:25.731+0000


In [28]:
ForumHasMember.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1782 entries, 0 to 1781
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Forum.id   1782 non-null   int64 
 1   Person.id  1782 non-null   int64 
 2   joinDate   1782 non-null   object
dtypes: int64(2), object(1)
memory usage: 41.9+ KB


In [42]:
forum_member = ForumHasMember['Forum.id'].unique()

In [52]:
ForumHasMember[ForumHasMember['Forum.id'] == "9895604649984"]

,Forum.id,Person.id,joinDate


In [43]:
forum_ids = Forum['id'].unique()

In [44]:
unique_id = []
for i in forum_member:
    if i not in forum_ids:
        unique_id.append(i)

In [45]:
len(unique_id)

0

In [29]:
ForumHasMember['joinDate'] = pd.to_datetime(ForumHasMember['joinDate'], errors='coerce')

In [53]:
import psycopg2
from psycopg2 import Error

# Assuming ForumHasMember is your DataFrame containing data to be inserted

# Insert query
query = """INSERT INTO ForumHasMember (
    ForumID, 
    PersonID,
    JoinDate) 
    VALUES (%s, %s, %s)"""

# Connection parameters
connection_params = {
    "dbname": "social_2",
    "user": "postgres",
    "password": "password",
    "host": "localhost",
    "port": "5432"
}

try:
    # Connect to the database
    connection = psycopg2.connect(**connection_params)
    
    # Create cursor
    with connection.cursor() as cursor:
        # Iterate over rows in the DataFrame and insert data into the table
        for index, row in ForumHasMember.iterrows():
            # Extract values from the DataFrame
            ForumID_value = row['Person.id']
            PersonID_value = row['Forum.id']
            JoinDate_value = row['joinDate']

            try:
                # Execute the insert query
                cursor.execute(query, (ForumID_value, PersonID_value, JoinDate_value))
                # Commit the transaction
                connection.commit()
            except psycopg2.errors.ForeignKeyViolation as e:
                # Skip conflicting data and print the error message
                print(f"Skipping conflicting data. Error: {e}")
            except Error as e:
                # Handle other database errors
                print(f"Error executing query: {e}")
                connection.rollback()  # Rollback the transaction
            
except psycopg2.Error as error:
    # Handle connection errors
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        connection.close()


Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(9895604649984) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(15393162788888) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(16492674416689) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(2199023255633) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(3298534883392) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(2199023255633) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update o

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(12094627905563) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(12094627905567) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(13194139533352) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or updat

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(8796093022251) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(9895604650020) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(9895604650020) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update o

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(13194139533405) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(10995116277764) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(3298534883405) is not present in table "forum".

Error executing query: current transaction is aborted, commands ignored until end of transaction block

Skipping conflicting data. Error: insert or update

In [38]:
#Insert into forum has member
query = """INSERT INTO ForumHasMember (
    ForumID, 
    PersonID,
    JoinDate) 
    VALUES (%s, %s, %s)"""


# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in ForumHasMember.iterrows():
        # Extract values from the DataFrame
        ForumID_value = row['Person.id']
        PersonID_value = row['Forum.id']
        JoinDate_value = row['joinDate']

         
        
        cursor.execute(query, (ForumID_value,
                               PersonID_value,
                              JoinDate_value))
        connection.commit()

# Close the database connection
connection.close()

ForeignKeyViolation: insert or update on table "forumhasmember" violates foreign key constraint "forumhasmember_forumid_fkey"
DETAIL:  Key (forumid)=(9895604649984) is not present in table "forum".


In [32]:
Forum[Forum['id'] == 9895604649984]

,id,title,creationDate,moderator


The ForumHasMember data throws a foreign key constraint, I will help you to solve these tomorrow

# Tag Table

In [3]:
Tag.head()

,id,name,url
0,0,Hamid_Karzai,http://dbpedia.org/resource/Hamid_Karzai
1,1,Rumi,http://dbpedia.org/resource/Rumi
2,2,Mahmud_of_Ghazni,http://dbpedia.org/resource/Mahmud_of_Ghazni
3,3,Abbas_I_of_Persia,http://dbpedia.org/resource/Abbas_I_of_Persia
4,4,Humayun,http://dbpedia.org/resource/Humayun


In [7]:
Tag['name'] = Tag['name'].astype(str)

In [10]:

query = """INSERT INTO Tag (
    TagID,
    name) 
    VALUES (%s, %s)"""


# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in Tag.iterrows():
        # Extract values from the DataFrame
        tagid_value = row['id']
        name_value = row['name']
        
        
         
        
        cursor.execute(query, (tagid_value, name_value))
        connection.commit()

# Close the database connection
connection.close()

## Tag Class Table

In [12]:
TagClass.head()

,id,name,url
0,349,OfficeHolder,http://dbpedia.org/ontology/OfficeHolder
1,211,Person,http://dbpedia.org/ontology/Person
2,239,Agent,http://dbpedia.org/ontology/Agent
3,0,Thing,http://www.w3.org/2002/07/owl#Thing
4,98,Monarch,http://dbpedia.org/ontology/Monarch


In [13]:
TagClass['id'] = TagClass['id'].astype(int)
TagClass['name'] = TagClass['name'].astype(str) 

In [15]:

query = """INSERT INTO TagClass (
    TagClassID,
    name) 
    VALUES (%s, %s)"""


# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in TagClass.iterrows():
        # Extract values from the DataFrame
        tagclassid_value = row['id']
        name_value = row['name']
        
        
         
        
        cursor.execute(query, (tagclassid_value, name_value))
        connection.commit()

# Close the database connection
connection.close()

In [17]:
TagHasType.head()

,Tag.id,TagClass.id
0,0,349
1,1,211
2,2,98
3,3,336
4,4,336


In [25]:
TagHasType['Tag.id'] = TagHasType['Tag.id'].astype('int64')
TagHasType['TagClass.id'] = TagHasType['TagClass.id'].astype('int64')

In [30]:
TagHasType.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16080 entries, 0 to 16079
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Tag.id       16080 non-null  int64
 1   TagClass.id  16080 non-null  int64
dtypes: int64(2)
memory usage: 251.4 KB


In [31]:

from psycopg2.extensions import register_adapter, AsIs
register_adapter(np.int64, AsIs)

query = """INSERT INTO TagHasType (
    TagID,
    TagClassID) 
    VALUES (%s, %s)"""


# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in TagHasType.iterrows():
        # Extract values from the DataFrame
        tagid_value = row['Tag.id']
        tagclassid_value = row['TagClass.id']
        
        
        
         
        
        cursor.execute(query, (tagid_value, tagclassid_value))
        connection.commit()

# Close the database connection
connection.close()

## Post Table

In [36]:
Post.head()

,id,imageFile,creationDate,locationIP,browserUsed,language,content,length,creator,Forum.id,place
0,128849018880,photo128849018880.jpg,2012-09-29T01:17:21.683+0000,175.111.0.55,Firefox,NaN,NaN,0,12094627905604,128849018882,54
1,128849018881,photo128849018881.jpg,2012-09-29T01:17:22.683+0000,175.111.0.55,Firefox,NaN,NaN,0,12094627905604,128849018882,54
2,128849018882,photo128849018882.jpg,2012-09-29T01:17:23.683+0000,175.111.0.55,Firefox,NaN,NaN,0,12094627905604,128849018882,54
3,128849018883,photo128849018883.jpg,2012-09-29T01:17:24.683+0000,175.111.0.55,Firefox,NaN,NaN,0,12094627905604,128849018882,54
4,128849018884,photo128849018884.jpg,2012-09-29T01:17:25.683+0000,175.111.0.55,Firefox,NaN,NaN,0,12094627905604,128849018882,54


In [37]:
Post.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7213 entries, 0 to 7212
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            7213 non-null   int64 
 1   imageFile     7001 non-null   object
 2   creationDate  7213 non-null   object
 3   locationIP    7213 non-null   object
 4   browserUsed   7213 non-null   object
 5   language      212 non-null    object
 6   content       212 non-null    object
 7   length        7213 non-null   int64 
 8   creator       7213 non-null   int64 
 9   Forum.id      7213 non-null   int64 
 10  place         7213 non-null   int64 
dtypes: int64(5), object(6)
memory usage: 620.0+ KB


## Post Has Tag

In [33]:
PostHasTag.head()

,Post.id,Tag.id
0,68719476859,6
1,68719476859,1199
2,128849019004,6
3,68719476861,6
4,111669149822,6


In [34]:
PostHasTag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Post.id  752 non-null    int64
 1   Tag.id   752 non-null    int64
dtypes: int64(2)
memory usage: 11.9 KB


In [35]:
query = """INSERT INTO PostHasTag (
    PostID,
    TagID) 
    VALUES (%s, %s)"""


# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in PostHasTag.iterrows():
        # Extract values from the DataFrame
        postid_value = row['Post.id']
        tagid_value = row['Tag.id']
        
        
        
         
        
        cursor.execute(query, (postid_value, tagid_value))
        connection.commit()

# Close the database connection
connection.close()

ForeignKeyViolation: insert or update on table "posthastag" violates foreign key constraint "posthastag_postid_fkey"
DETAIL:  Key (postid)=(68719476859) is not present in table "post".


## Place (city, country, continent) Tables

In [55]:
Place.head()

,id,name,url,type,isPartOf
0,0,India,http://dbpedia.org/resource/India,country,1460.0
1,1,China,http://dbpedia.org/resource/China,country,1460.0
2,2,Angola,http://dbpedia.org/resource/Angola,country,1461.0
3,3,Austria,http://dbpedia.org/resource/Austria,country,1462.0
4,4,Azerbaijan,http://dbpedia.org/resource/Azerbaijan,country,1460.0


In [61]:
Place.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1466 entries, 0 to 1465
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        1466 non-null   int64  
 1   name      1466 non-null   object 
 2   url       1466 non-null   object 
 3   type      1466 non-null   object 
 4   isPartOf  1460 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 57.4+ KB


In [57]:
country_subset = Place[Place['type'] == 'country']

In [58]:
city_subset = Place[Place['type'] == 'city']

In [59]:
continent_subset = Place[Place['type'] == 'continent']